SUPPLEMENTARY MATERIAL TABLES 2 AND 3

In [2]:
import os
import json
import numpy as np
from glob import glob
import pandas as pd

# Define variables
tools = ["DIFFAB", "ADESIGN", "dyMEAN"]
base_dir = "./NanoDesigner/Tool_assesment_results"


In [ ]:
# Tool directories
tool_dirs = {
    "DIFFAB": os.path.join(base_dir, "DIFFAB_ASSESMENT"),
    "ADESIGN": os.path.join(base_dir, "ADESIGN_ASSESMENT"),
    "dyMEAN": os.path.join(base_dir, "dyMEAN_ASSESMENT")
}

# Training configurations
training_configs = {
    "Config1": "Nanobody",
    "Config2": "Nanobody_Antibody", 
    "Config3": "Antibody"
}

# Clustering configurations
clustering_configs = {
    "CDRH3": ["20", "30", "40"],
    "Ag": ["60", "80", "95"]
}

# Method name mapping
method_mapping = {
    'DIFFAB': 'DiffAb',
    'ADESIGN': 'ADesigner',
    'dyMEAN': 'dyMEAN'
}

def find_metrics_files():
    """Find all metrics files with configuration parsing"""
    file_paths = {}
    
    for tool in tools:
        tool_dir = tool_dirs[tool]
        file_paths[tool] = []
        
        # Look for all possible clustering configurations
        for cluster_type, thresholds in clustering_configs.items():
            for threshold in thresholds:
                for config_name, config_folder in training_configs.items():
                    # Construct the expected directory pattern
                    cluster_dir = f"{config_folder}_clustered_{cluster_type}_{threshold}"
                    search_pattern = os.path.join(tool_dir, cluster_dir, "**", "metrics_fold_*.json")
                    
                    # Find files matching this pattern
                    json_files = glob(search_pattern, recursive=True)
                    
                    for json_file in json_files:
                        # Extract fold number from filename
                        fold_num = json_file.split('metrics_fold_')[-1].split('.json')[0]
                        
                        file_info = {
                            'path': json_file,
                            'tool': tool,
                            'config': config_name,
                            'cluster_type': cluster_type,
                            'threshold': threshold,
                            'fold': fold_num
                        }
                        file_paths[tool].append(file_info)
    
    return file_paths

def load_metrics_data(file_paths):
    """Load metrics data from JSON files, prioritizing fold_0, then fold_1"""
    data = []
    
    for tool, files in file_paths.items():
        # Group files by configuration
        config_groups = {}
        for file_info in files:
            key = f"{file_info['config']}_{file_info['cluster_type']}_{file_info['threshold']}"
            if key not in config_groups:
                config_groups[key] = []
            config_groups[key].append(file_info)
        
        # For each configuration, select fold_0 or fold_1
        for config_key, config_files in config_groups.items():
            selected_file = None
            
            # Try to find fold_0 first
            for file_info in config_files:
                if file_info['fold'] == '0':
                    selected_file = file_info
                    break
            
            # If fold_0 not found, try fold_1
            if selected_file is None:
                for file_info in config_files:
                    if file_info['fold'] == '1':
                        selected_file = file_info
                        break
            
            # If still not found, take the first available
            if selected_file is None:
                selected_file = config_files[0]
            
            try:
                with open(selected_file['path'], 'r') as f:
                    # Handle JSONL format - multiple JSON objects per line
                    json_data = [json.loads(line) for line in f if line.strip()]
                    
                    # Process ALL entries, not just the first one
                    for entry_idx, metrics in enumerate(json_data):
                        # Add configuration info to metrics
                        metrics_entry = {
                            'Tool': method_mapping[selected_file['tool']],
                            'Config': selected_file['config'],
                            'Cluster_Type': selected_file['cluster_type'],
                            'Threshold': selected_file['threshold'],
                            'Fold': selected_file['fold'],
                            'File_Path': selected_file['path'],
                            'Entry_Index': entry_idx
                        }
                        
                        # Add all metrics
                        metrics_entry.update(metrics)
                        data.append(metrics_entry)
                        
            except Exception as e:
                print(f"Error loading {selected_file['path']}: {e}")
    
    return data

def generate_latex_table(data, cluster_type, caption="TBD", label_suffix="clustering"):
    """
    Generate LaTeX table for antibody design tool performance metrics
    
    Parameters:
    - data: List of dictionaries containing metrics data
    - cluster_type: Either "CDRH3" or "Ag"
    - caption: Table caption
    - label_suffix: Suffix for table label
    
    Returns:
    - String containing LaTeX table code
    """
    
    # Filter data for the specific cluster type
    filtered_data = [d for d in data if d.get('Cluster_Type') == cluster_type]
    
    if not filtered_data:
        return ""
    
    # Convert to DataFrame for easier manipulation
    df = pd.DataFrame(filtered_data)
    
    # Define the configuration mapping and order
    config_mapping = {
        'Config2': 'Nb+Ab',  # Nanobody_Antibody
        'Config1': 'Nanobodies',  # Nanobody
        'Config3': 'Antibodies'   # Antibody
    }
    
    # Define thresholds based on cluster type
    if cluster_type == "CDRH3":
        thresholds = ["20", "30", "40"]
        threshold_suffix = "%"
    else:  # Ag
        thresholds = ["60", "80", "95"]
        threshold_suffix = "%"
    
    # Define tools order
    tools_order = ['ADesigner', 'DiffAb', 'dyMEAN']
    
    # Define metric columns and their formatting
    metric_columns = {
        'AAR H3': ('AAR\\\\H3', '↑', '.3f'),
        'RMSD(CA) aligned': ('RMSD', '↓', '.3f'),
        'RMSD(CA) CDRH3 aligned': ('RMSD\\\\CDRH3', '↓', '.3f'),
        'TMscore': ('TM-\\\\score', '↑', '.3f'),
        'LDDT': ('LDDT', '↑', '.3f'),
        'FoldX_dG': ('ΔG', '↓', '.3f'),
        'FoldX_ddG': ('ΔΔG', '↓', '.3f'),
        'final_num_clashes': ('Clashes', '↓', 'd'),
        'DockQ': ('DockQ', '↑', '.3f'),
        'Success_Rate': ('Success\\\\Rate (%)', '↑', '.1f')
    }
    
    # Start building the LaTeX table
    latex_code = []
    
    # Table header
    latex_code.append("\\clearpage")
    latex_code.append("\\begin{table}[h]")
    latex_code.append("\\centering")
    latex_code.append("\\footnotesize")
    latex_code.append("\\centering")
    latex_code.append(f"\\caption{{{caption}}}")
    latex_code.append(f"\\label{{tab:{cluster_type.lower()}_{label_suffix}}}")
    
    # Column specification
    num_metrics = len(metric_columns)
    col_spec = f"@{{}}lll*{{{num_metrics}}}{{c}}@{{}}"
    latex_code.append(f"\\begin{{tabular}}{{{col_spec}}}")
    
    # Top rule
    latex_code.append("\\toprule")
    
    # Header row
    header_parts = [
        "\\rotatebox{90}{\\textbf{}}",
        "\\rotatebox{90}{\\textbf{}}",
        "\\textbf{Tool}",
        "\\makecell{\\textbf{AAR}\\\\\\textbf{H3} $\\uparrow$}",
        "\\makecell{\\textbf{RMSD}\\\\\\textbf{}  $\\downarrow$}",
        "\\makecell{\\textbf{RMSD}\\\\\\textbf{CDRH3} $\\downarrow$}",
        "\\makecell{\\textbf{TM-}\\\\\\textbf{score} $\\uparrow$}",
        "\\makecell{\\textbf{LDDT} $\\uparrow$}",
        "\\makecell{\\textbf{$\\Delta$G} $\\downarrow$}",
        "\\makecell{\\textbf{$\\Delta\\Delta$G} $\\downarrow$}",
        "\\makecell{\\textbf{Clashes} $\\downarrow$}",
        "\\makecell{\\textbf{DockQ} $\\uparrow$}",
        "\\makecell{\\textbf{Success}\\\\\\textbf{Rate (\\%)} $\\uparrow$}"
    ]
    
    latex_code.append(" & ".join(header_parts) + " \\\\")
    latex_code.append("\\cmidrule(r){1-3} \\cmidrule(l){4-13}")
    
    # Data rows
    for config_idx, (config_key, config_display) in enumerate(config_mapping.items()):
        config_data = df[df['Config'] == config_key]
        
        if config_data.empty:
            continue
        
        # Calculate number of rows for this configuration (3 tools × number of thresholds)
        num_rows = len(thresholds) * len(tools_order)
        
        first_row = True
        for threshold_idx, threshold in enumerate(thresholds):
            threshold_data = config_data[config_data['Threshold'] == threshold]
            
            for tool_idx, tool in enumerate(tools_order):
                tool_data = threshold_data[threshold_data['Tool'] == tool]
                
                # Prepare row data
                row_parts = []
                
                # Configuration column (only for first row of each config)
                if first_row:
                    row_parts.append(f"\\multirow{{{num_rows}}}{{*}}{{\\rotatebox[origin=c]{{90}}{{\\textbf{{{config_display}}}}}}}")
                    first_row = False
                else:
                    row_parts.append("")
                
                # Threshold column (only for first tool of each threshold)
                if tool_idx == 0:
                    threshold_clean = str(threshold).replace('%', '')
                    row_parts.append(f"\\multirow{{3}}{{*}}{{\\rotatebox[origin=c]{{90}}{{{threshold_clean}\\%}}}}")
                else:
                    row_parts.append("")
                
                # Tool name
                row_parts.append(tool)
                
                # Metric values
                if not tool_data.empty:
                    for col_key, (_, _, fmt) in metric_columns.items():
                        if col_key == 'Success_Rate':
                            # Calculate success rate across ALL entries for this tool+config+threshold
                            all_ddg_values = []
                            for _, row in tool_data.iterrows():
                                ddg_val = row.get('FoldX_ddG', np.nan)
                                if pd.notna(ddg_val):
                                    all_ddg_values.append(ddg_val)
                            
                            if len(all_ddg_values) > 0:
                                ddg_array = np.array(all_ddg_values)
                                negative_count = np.sum(ddg_array < 0)
                                success_rate = (negative_count / len(ddg_array)) * 100
                                value = success_rate
                            else:
                                value = 0
                        elif col_key == 'final_num_clashes':
                            # For clashes, aggregate and round to integer
                            all_values = []
                            for _, row in tool_data.iterrows():
                                metric_val = row.get(col_key, np.nan)
                                if pd.notna(metric_val):
                                    all_values.append(metric_val)
                            
                            if len(all_values) > 0:
                                value = round(np.mean(all_values))
                            else:
                                value = 0
                        elif col_key == 'DockQ':
                            # For DockQ, aggregate with 1 decimal place
                            all_values = []
                            for _, row in tool_data.iterrows():
                                metric_val = row.get(col_key, np.nan)
                                if pd.notna(metric_val):
                                    all_values.append(metric_val)
                            
                            if len(all_values) > 0:
                                value = np.mean(all_values)
                            else:
                                value = 0
                        else:
                            # For other metrics, aggregate across all entries
                            all_values = []
                            for _, row in tool_data.iterrows():
                                metric_val = row.get(col_key, np.nan)
                                if pd.notna(metric_val):
                                    all_values.append(metric_val)
                            
                            if len(all_values) > 0:
                                value = np.mean(all_values)
                            else:
                                value = 0
                        
                        formatted_value = f"{value:{fmt}}"
                        row_parts.append(formatted_value)
                else:
                    # No data for this tool/threshold combination
                    for _ in metric_columns:
                        row_parts.append("0.000")
                
                latex_code.append(" & ".join(row_parts) + " \\\\")
            
            # Add cmidrule between thresholds (except for the last threshold of each config)
            if threshold_idx < len(thresholds) - 1:
                latex_code.append(f"\\cmidrule(lr){{2-{3+num_metrics}}}")
        
        # Add midrule between configurations (except for the last config)
        if config_idx < len(config_mapping) - 1:
            latex_code.append("\\midrule")
    
    # Bottom rule and table end
    latex_code.append("\\bottomrule")
    latex_code.append("\\end{tabular}")
    latex_code.append("\\end{table}")
    
    return "\n".join(latex_code)

def generate_all_tables(data):
    """
    Generate LaTeX tables for both CDRH3 and Ag clustering
    
    Parameters:
    - data: List of dictionaries containing metrics data
    
    Returns:
    - Dictionary with 'CDRH3' and 'Ag' keys containing LaTeX code
    """
    
    tables = {}
    
    # Generate CDRH3 clustering table
    tables['CDRH3'] = generate_latex_table(
        data, 
        'CDRH3', 
        caption="Performance metrics for antibody design tools with CDRH3 clustering",
        label_suffix="cdrh3_clustering"
    )
    
    # Generate Ag clustering table
    tables['Ag'] = generate_latex_table(
        data, 
        'Ag', 
        caption="Performance metrics for antibody design tools with Antigen clustering", 
        label_suffix="ag_clustering"
    )
    
    return tables

def main():
    """Main function to generate and display tables"""
    file_paths = find_metrics_files()
    data = load_metrics_data(file_paths)
    tables = generate_all_tables(data)
    
    print("="*80)
    print("CDRH3 CLUSTERING TABLE")
    print("="*80)
    print(tables['CDRH3'])
    
    print("\n" + "="*80)
    print("AG CLUSTERING TABLE") 
    print("="*80)
    print(tables['Ag'])
    
    return tables

# Run the main function
if __name__ == "__main__":
    tables = main()

CDRH3 CLUSTERING TABLE
\clearpage
\begin{table}[h]
\centering
\footnotesize
\centering
\caption{Performance metrics for antibody design tools with CDRH3 clustering}
\label{tab:cdrh3_cdrh3_clustering}
\begin{tabular}{@{}lll*{10}{c}@{}}
\toprule
\rotatebox{90}{\textbf{}} & \rotatebox{90}{\textbf{}} & \textbf{Tool} & \makecell{\textbf{AAR}\\\textbf{H3} $\uparrow$} & \makecell{\textbf{RMSD}\\\textbf{}  $\downarrow$} & \makecell{\textbf{RMSD}\\\textbf{CDRH3} $\downarrow$} & \makecell{\textbf{TM-}\\\textbf{score} $\uparrow$} & \makecell{\textbf{LDDT} $\uparrow$} & \makecell{\textbf{$\Delta$G} $\downarrow$} & \makecell{\textbf{$\Delta\Delta$G} $\downarrow$} & \makecell{\textbf{Clashes} $\downarrow$} & \makecell{\textbf{DockQ} $\uparrow$} & \makecell{\textbf{Success}\\\textbf{Rate (\%)} $\uparrow$} \\
\cmidrule(r){1-3} \cmidrule(l){4-13}
\multirow{9}{*}{\rotatebox[origin=c]{90}{\textbf{Nb+Ab}}} & \multirow{3}{*}{\rotatebox[origin=c]{90}{20\%}} & ADesigner & 0.362 & 1.000 & 2.282 & 0.969 & 0.83

TABLE 1 MAIN TEXT

In [ ]:
import os
import json
import numpy as np
from glob import glob
import pandas as pd
from scipy import stats

# Define variables
tools = ["DIFFAB", "ADESIGN"]  # Only focus on these two
base_dir = "./NanoDesigner/Tool_assesment_results"


In [6]:
# Training configurations - only Config2
training_configs = {
    "Config2": "Nanobody_Antibody"
}

# Clustering configurations
clustering_configs = {
    "CDRH3": ["40"],
    "Ag": ["60"]
}

# Method name mapping
method_mapping = {
    'DIFFAB': 'DiffAb',
    'ADESIGN': 'ADesigner'
}

def find_metrics_files():
    """Find all metrics files for specified clustering configurations"""
    file_paths = {}
    
    for tool in tools:
        tool_dir = tool_dirs[tool]
        file_paths[tool] = []
        
        # Loop through ALL clustering configurations
        for cluster_type, thresholds in clustering_configs.items():
            for threshold in thresholds:
                config_name = "Config2"
                config_folder = training_configs[config_name]
                
                cluster_dir = f"{config_folder}_clustered_{cluster_type}_{threshold}"
                search_pattern = os.path.join(tool_dir, cluster_dir, "**", "metrics_fold_*.json")
                
                json_files = glob(search_pattern, recursive=True)
        
                for json_file in json_files:
                    # Extract fold number from filename
                    fold_num = json_file.split('metrics_fold_')[-1].split('.json')[0]
                    
                    file_info = {
                        'path': json_file,
                        'tool': tool,
                        'config': config_name,
                        'cluster_type': cluster_type,
                        'threshold': threshold,
                        'fold': fold_num
                    }
                    file_paths[tool].append(file_info)
    
    return file_paths

def load_metrics_data(file_paths):
    """Load metrics data from JSON files, gathering ALL folds"""
    data = []
    
    for tool, files in file_paths.items():
        print(f"Loading data for {tool}:")
        
        for file_info in files:
            try:
                with open(file_info['path'], 'r') as f:
                    # Handle JSONL format - multiple JSON objects per line
                    json_data = [json.loads(line) for line in f if line.strip()]
                    
                    # Process ALL entries
                    for entry_idx, metrics in enumerate(json_data):
                        # Add configuration info to metrics
                        metrics_entry = {
                            'Tool': method_mapping[file_info['tool']],
                            'Config': file_info['config'],
                            'Cluster_Type': file_info['cluster_type'],
                            'Threshold': file_info['threshold'],
                            'Fold': file_info['fold'],
                            'File_Path': file_info['path'],
                            'Entry_Index': entry_idx
                        }
                        
                        # Add all metrics
                        metrics_entry.update(metrics)
                        data.append(metrics_entry)
                
                print(f"  Fold {file_info['fold']}: {len(json_data)} entries from {file_info['path']}")
                        
            except Exception as e:
                print(f"Error loading {file_info['path']}: {e}")
    
    return data

def calculate_confidence_interval(values, confidence=0.95):
    """Calculate confidence interval for a list of values"""
    if len(values) == 0:
        return 0, 0, 0
    
    values = np.array(values)
    mean = np.mean(values)
    
    if len(values) == 1:
        return mean, 0, 0
    
    # Calculate standard error
    sem = stats.sem(values)
    # Calculate confidence interval
    ci = sem * stats.t.ppf((1 + confidence) / 2., len(values) - 1)
    
    return mean, ci, sem

def aggregate_metrics_by_tool_and_clustering(data):
    """Aggregate metrics by tool and clustering configuration across all folds"""
    
    # Define the metrics we want to analyze
    metric_columns = [
        'AAR H3',
        'RMSD(CA) aligned', 
        'RMSD(CA) CDRH3 aligned',
        'TMscore',
        'LDDT',
        'FoldX_dG',
        'FoldX_ddG',
        'final_num_clashes',
        'DockQ'
    ]
    
    results = {}
    
    # Get unique clustering configurations
    cluster_configs = set((d['Cluster_Type'], d['Threshold']) for d in data)
    print(f"Found clustering configurations: {cluster_configs}")
    
    for cluster_type, threshold in cluster_configs:
        config_key = f"{cluster_type}_{threshold}"
        results[config_key] = {}
        
        print(f"\n=== Processing {cluster_type} {threshold}% ===")
        
        # Filter data for this clustering configuration
        cluster_data = [d for d in data if d['Cluster_Type'] == cluster_type and d['Threshold'] == threshold]
        
        for tool in ['DiffAb', 'ADesigner']:
            tool_data = [d for d in cluster_data if d['Tool'] == tool]
            print(f"\nProcessing {tool}: {len(tool_data)} total entries")
            
            # Group by fold to see distribution
            folds = {}
            for entry in tool_data:
                fold = entry['Fold']
                if fold not in folds:
                    folds[fold] = []
                folds[fold].append(entry)
            
            print(f"  Available folds: {list(folds.keys())}")
            for fold, entries in folds.items():
                print(f"    Fold {fold}: {len(entries)} entries")
            
            tool_results = {}
            
            for metric in metric_columns:
                # Collect all values for this metric across all folds
                all_values = []
                for entry in tool_data:
                    if metric in entry and pd.notna(entry[metric]):
                        all_values.append(entry[metric])
                
                if len(all_values) > 0:
                    mean, ci, sem = calculate_confidence_interval(all_values)
                    tool_results[metric] = {
                        'mean': mean,
                        'ci': ci,
                        'sem': sem,
                        'count': len(all_values),
                        'values': all_values
                    }
                    print(f"  {metric}: {len(all_values)} values, mean={mean:.3f}, ci=±{ci:.3f}")
                else:
                    tool_results[metric] = {
                        'mean': 0,
                        'ci': 0,
                        'sem': 0,
                        'count': 0,
                        'values': []
                    }
                    print(f"  {metric}: No valid values found")
            
            # Calculate success rate (percentage of negative FoldX_ddG values)
            ddg_values = [entry['FoldX_ddG'] for entry in tool_data 
                         if 'FoldX_ddG' in entry and pd.notna(entry['FoldX_ddG'])]
            
            if len(ddg_values) > 0:
                success_count = sum(1 for val in ddg_values if val < 0)
                success_rate = (success_count / len(ddg_values)) * 100
                
                # For success rate, we can calculate CI using binomial proportion
                p = success_count / len(ddg_values)
                se = np.sqrt(p * (1 - p) / len(ddg_values))
                ci_prop = 1.96 * se * 100  # 95% CI in percentage
                
                tool_results['Success_Rate'] = {
                    'mean': success_rate,
                    'ci': ci_prop,
                    'sem': se * 100,
                    'count': len(ddg_values),
                    'success_count': success_count
                }
                print(f"  Success Rate: {success_count}/{len(ddg_values)} = {success_rate:.1f}% ± {ci_prop:.1f}%")
            else:
                tool_results['Success_Rate'] = {
                    'mean': 0,
                    'ci': 0,
                    'sem': 0,
                    'count': 0,
                    'success_count': 0
                }
            
            results[config_key][tool] = tool_results
    
    return results

def format_metric_value(mean, ci, metric_name, is_best=False, is_second=False):
    """Format metric value with confidence interval and styling"""
    
    # Format based on metric type
    if metric_name == 'final_num_clashes':
        formatted = f"{mean:.0f} $\\pm$ {ci:.0f}"
    elif metric_name == 'Success_Rate':
        formatted = f"{mean:.1f} $\\pm$ {ci:.1f}"
    else:
        formatted = f"{mean:.3f} $\\pm$ {ci:.3f}"
    
    # Apply styling
    if is_best:
        formatted = f"\\textbf{{{formatted}}}"
    elif is_second:
        formatted = f"\\underline{{{formatted}}}"
    
    return formatted

def determine_best_values_for_clustering(cluster_results, metric_name, higher_better=True):
    """Determine which tool has best and second best values for a specific clustering"""
    
    values = []
    for tool in ['DiffAb', 'ADesigner']:
        if tool in cluster_results and metric_name in cluster_results[tool]:
            values.append((tool, cluster_results[tool][metric_name]['mean']))
    
    if len(values) < 2:
        return {}, {}
    
    # Sort based on whether higher or lower is better
    values.sort(key=lambda x: x[1], reverse=higher_better)
    
    best_tool = values[0][0]
    second_tool = values[1][0]
    
    return {best_tool: True}, {second_tool: True}

def generate_latex_table_for_clustering(results, cluster_config):
    """Generate LaTeX table for a specific clustering configuration"""
    
    if cluster_config not in results:
        return f"% No data found for {cluster_config}"
    
    cluster_results = results[cluster_config]
    
    # Define metrics with their display names and whether higher is better
    metrics_info = {
        'AAR H3': ('AAR H3', True),
        'RMSD(CA) aligned': ('RMSD', False),
        'RMSD(CA) CDRH3 aligned': ('RMSD CDRH3', False),
        'TMscore': ('TMscore', True),
        'LDDT': ('LDDT', True),
        'FoldX_dG': ('$\\Delta G$', False),
        'FoldX_ddG': ('$\\Delta \\Delta G$', False),
        'final_num_clashes': ('Clashes', False),
        'DockQ': ('DockQ', True),
        'Success_Rate': ('Success Rate \\%', True)
    }
    
    # Extract cluster info for caption
    if 'CDRH3' in cluster_config:
        caption_text = f"Performance metrics for antibody design tools (CDRH3 {cluster_config.split('_')[1]}\\% clustering, Config2)"
        label_text = f"cdrh3_{cluster_config.split('_')[1]}_config2_metrics"
    else:
        caption_text = f"Performance metrics for antibody design tools (Ag {cluster_config.split('_')[1]}\\% clustering, Config2)"
        label_text = f"ag_{cluster_config.split('_')[1]}_config2_metrics"
    
    latex_lines = []
    
    # Table header
    latex_lines.extend([
        "\\begin{table}[h]",
        "\\centering",
        f"\\caption{{{caption_text}}}",
        f"\\label{{tab:{label_text}}}",
        "\\begin{tabular}{l c c}",
        "\\toprule",
        "\\textbf{Metric} & \\textbf{DiffAb} & \\textbf{ADesigner} \\\\",
        "\\midrule"
    ])
    
    # Generate rows for each metric
    for metric_key, (display_name, higher_better) in metrics_info.items():
        
        # Determine best and second best for this clustering
        best_tools, second_tools = determine_best_values_for_clustering(cluster_results, metric_key, higher_better)
        
        # Format values for each tool
        diffab_formatted = format_metric_value(
            cluster_results['DiffAb'][metric_key]['mean'],
            cluster_results['DiffAb'][metric_key]['ci'],
            metric_key,
            is_best=best_tools.get('DiffAb', False),
            is_second=second_tools.get('DiffAb', False)
        )
        
        adesigner_formatted = format_metric_value(
            cluster_results['ADesigner'][metric_key]['mean'],
            cluster_results['ADesigner'][metric_key]['ci'],
            metric_key,
            is_best=best_tools.get('ADesigner', False),
            is_second=second_tools.get('ADesigner', False)
        )
        
        # Add direction indicator
        direction = "$\\uparrow$" if higher_better else "$\\downarrow$"
        display_name_with_direction = f"{display_name} {direction}"
        
        latex_lines.append(f"{display_name_with_direction} & {diffab_formatted} & {adesigner_formatted} \\\\")
    
    # Table footer
    latex_lines.extend([
        "\\bottomrule",
        "\\end{tabular}",
        "\\end{table}"
    ])
    
    return "\n".join(latex_lines)

def main():
    """Main function to generate the focused analysis"""
    print("Finding metrics files for multiple clustering configurations...")
    file_paths = find_metrics_files()
    
    # Debug: Print found files
    total_files = sum(len(files) for files in file_paths.values())
    print(f"Found {total_files} total metrics files")
    for tool, files in file_paths.items():
        print(f"  {tool}: {len(files)} files")
        
        # Group by clustering configuration
        cluster_groups = {}
        for file_info in files:
            key = f"{file_info['cluster_type']}_{file_info['threshold']}"
            if key not in cluster_groups:
                cluster_groups[key] = []
            cluster_groups[key].append(file_info)
        
        for cluster_key, cluster_files in cluster_groups.items():
            print(f"    {cluster_key}: {len(cluster_files)} files")
            for file_info in cluster_files:
                print(f"      Fold {file_info['fold']}: {file_info['path']}")
    
    if total_files == 0:
        print("No files found! Check your paths and configuration.")
        return
    
    print("\nLoading metrics data...")
    data = load_metrics_data(file_paths)
    print(f"Loaded data for {len(data)} total entries")
    
    if len(data) == 0:
        print("No data loaded! Check your file formats.")
        return
    
    # Check what clustering configurations we actually have data for
    available_configs = set((d['Cluster_Type'], d['Threshold']) for d in data)
    print(f"Available clustering configurations: {available_configs}")
    
    # Check what tools we actually have data for
    available_tools = set(d['Tool'] for d in data)
    print(f"Available tools in data: {available_tools}")
    
    print("\nAggregating metrics by tool and clustering configuration...")
    results = aggregate_metrics_by_tool_and_clustering(data)
    
    print("\nGenerating LaTeX tables...")
    
    # Generate tables for each clustering configuration
    tables = {}
    for cluster_config in results.keys():
        latex_table = generate_latex_table_for_clustering(results, cluster_config)
        tables[cluster_config] = latex_table
        
        print(f"\n" + "="*80)
        print(f"LATEX TABLE FOR {cluster_config.upper()} CLUSTERING, NB + AB")
        print("="*80)
        print(latex_table)
        
        # # Save individual table to file
        # filename = f"{cluster_config.lower()}_config2_table.tex"
        # with open(filename, "w") as f:
        #     f.write(latex_table)
        # print(f"\nTable saved to {filename}")
    
    return results, tables

# Run the analysis
if __name__ == "__main__":
    results, tables = main()

Finding metrics files for multiple clustering configurations...
Found 33 total metrics files
  DIFFAB: 13 files
    CDRH3_40: 8 files
      Fold 4: /ibex/user/rioszemm/NANODESIGER_TOOL_ASSESMENT_2025/DIFFAB_ASSESMENT/Nanobody_Antibody_clustered_CDRH3_40/metrics_per_fold/metrics_fold_4.json
      Fold 1: /ibex/user/rioszemm/NANODESIGER_TOOL_ASSESMENT_2025/DIFFAB_ASSESMENT/Nanobody_Antibody_clustered_CDRH3_40/metrics_per_fold/metrics_fold_1.json
      Fold 7: /ibex/user/rioszemm/NANODESIGER_TOOL_ASSESMENT_2025/DIFFAB_ASSESMENT/Nanobody_Antibody_clustered_CDRH3_40/metrics_per_fold/metrics_fold_7.json
      Fold 2: /ibex/user/rioszemm/NANODESIGER_TOOL_ASSESMENT_2025/DIFFAB_ASSESMENT/Nanobody_Antibody_clustered_CDRH3_40/metrics_per_fold/metrics_fold_2.json
      Fold 6: /ibex/user/rioszemm/NANODESIGER_TOOL_ASSESMENT_2025/DIFFAB_ASSESMENT/Nanobody_Antibody_clustered_CDRH3_40/metrics_per_fold/metrics_fold_6.json
      Fold 0: /ibex/user/rioszemm/NANODESIGER_TOOL_ASSESMENT_2025/DIFFAB_ASSESMEN